<a href="https://colab.research.google.com/github/hochschule-pforzheim/project-st23-team-f23/blob/main/1_Cleansing_Join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Laden der wichtigsten Komponenten**

> Zuerst muss Pandas (unter der Abkürzung pd) und der Datensatz der csv-Datei ([hotel_orig.csv](https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv)) als 'hotel_orig' in das Jupiter Notebook geladen und als Dataframe über die Funktion '`read.csv()`' gespeichert werden. Die Spalte 'reservation_status_date' wird mit dem 'parse_dates' Befehl als Dateityp Dateum übergeben.


In [ ]:
import pandas as pd
hotel_orig = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv', parse_dates=['reservation_status_date'])

> Um einen ersten Eindruck über den Datensatz sowie den Aufbau des Dataframes zu bekommen, wird das Dataframe ausgegeben. Um mehr als die nur die ersten und letzten 5 Zeilen angezeigt zu bekommen, setzen wir die minimale Anzahl an gezeigten Reihen mit `pd.set_option('display.min_rows', 50)` auf 50 und mit `pd.set_option('display.max_columns', 500)` können wir alle Variablen im Dataframe ausgeben lassen.


In [ ]:
# set options to show more rows and columns for exploration
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
hotel_orig

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.00,0,0,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,NaN,0,Transient,103.00,0,1,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.00,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.50,0,0,Canceled,2015-04-22


## **Erste relevante Strukturanpassungen als Vorbereitung für spätere Analysen**







> Da das Datum der Buchung in 3 verschiedenen Spalten (arrival_date_year, arrival_date_month, arrival_date_day_of_month) im Datensatz hinterlegt ist, werden diese Spalten zur einfacheren Auswertung vereint. Mit dem Befehl des Panda Paketes 'to_datetime' wird das anschließend, in Klammern stehende, Argument zu einem Datum transformiert. Es werden die drei einzelen Spalten zu einem String transformiert (`'astype(str)'`) und anschließend mit einem - voneinander getrennt, hintereinander weg in eine neue Spalte namens 'arrival_date' im Dataframe hinterlegt. Diese hat das Format yyyy-mm-dd.






In [ ]:
# melt various date columns to one readable date-column
hotel_orig['arrival_date'] = pd.to_datetime(hotel_orig.arrival_date_year.astype(str) + '-' + hotel_orig.arrival_date_month.astype(str) + '-' + hotel_orig.arrival_date_day_of_month.astype(str))

> Um die neue Spalte 'arrival_date' nicht als letzte Spalte des Dataframes anzuhängen, ändern wir über die Funktion '`insert()`' die Position der neuen Spalte zum Index 3 der x-Achse (der Achse, auf der die Spalten liegen). Dafür wird die Spalte, die ursprünglich am Ende eingefügt war, verwendet und am ursprünglichen Ort entfernt.

In [ ]:
# change order of columns (put new arrival_date-column to the beginning)
hotel_orig.insert(3, 'arrival_date', hotel_orig.pop('arrival_date'))



> Als ID in dem Dataframe wird nun die Spalte 'night_index' erstellt, welche einfach die Reihen von oben nach unten durchzählt und so einen eindeutigen identifier bereitstellt. Dieser Idex wird noch an Position 1 gesetzt.



In [ ]:
hotel_orig['night_index'] = range(len(hotel_orig))
hotel_orig.loc[:, 'night_index'] = range(len(hotel_orig))

In [ ]:
hotel_orig.insert(0, 'night_index', hotel_orig.pop('night_index'))



> Für spätere Analysen scheint eine noch nicht vorhandene Zeile im Datensatz wichtig, die Summe der Nächte, für jede Buchung. Hierfür werden die gebuchten Nächte in der Woche und die gebuchten Nächte am Wochenende aufsummiert und in einer neuen Spalte namens 'stay_nights_sum' abgespeichert.



In [ ]:
hotel_orig['stay_nights_sum'] = hotel_orig['stays_in_week_nights'] + hotel_orig['stays_in_weekend_nights'] 

> Mit der Funktion '`value_counts()`' kann im nächsten Schritt bestimmt werden, wie häufig ein gleicher Wert in einer bestimmten Spalte angegeben wurde. So haben die meisten buchenden Personen (27643) für 2 Nächte gebucht. Für sehr lange Zeiträume, beispielsweise für 34, 57 oder 43 Nächte, haben jeweils nur eine Person gebucht.


In [ ]:
hotel_orig.stay_nights_sum.value_counts() #.describe auch angucken

2     27643
3     27076
1     21020
4     17383
7      8655
5      7784
6      3857
8      1161
10     1139
14      916
9       841
0       715
11      396
12      223
13      142
15       75
21       71
16       40
25       37
18       35
28       35
19       22
17       20
29       14
20       14
22       14
30       13
23        8
24        6
26        6
27        5
35        5
42        4
33        3
56        2
34        1
57        1
49        1
48        1
69        1
38        1
45        1
60        1
46        1
43        1
Name: stay_nights_sum, dtype: int64

Im nächsten Schritt wird der Datensatz um die Spalte 'departure_date' erweitert. Hierfür ist die Funktion timedelta des Paketes datetime notwendig. Eine lambda Funktion fügt der 'arrival_date'-Zeit den Wert der Spalte "stay_nights_sum" Tage hinzu und gibt das Ergebnis zurück.

In [ ]:
from datetime import timedelta

hotel_orig['departure_date'] = hotel_orig.apply(lambda row: row['arrival_date'] + timedelta(days=row['stay_nights_sum']), axis=1)

In [ ]:
hotel_orig.departure_date.describe()

count                  119390
unique                    806
top       2015-12-08 00:00:00
freq                      452
first     2015-07-01 00:00:00
last      2017-09-14 00:00:00
Name: departure_date, dtype: object

Die beiden Spalten 'departure_date' und 'stay_nights_sum' werden an passendere Positionen im Datensatz verschoben.

In [ ]:
hotel_orig.insert(5, 'departure_date', hotel_orig.pop('departure_date'))
hotel_orig.insert(12, 'stay_nights_sum', hotel_orig.pop('stay_nights_sum'))


> Das Dataframe kann nun durch Eingabe des gespeicherten Variablennamen und Ausführen der Codezelle mit den vorgenommenen Einstellungen und Änderungen angezeigt werden.



In [ ]:
# show datafrane
hotel_orig

,night_index,hotel,is_canceled,lead_time,arrival_date,departure_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stay_nights_sum,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,0,342,2015-07-01,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,1,Resort Hotel,0,737,2015-07-01,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,2,Resort Hotel,0,7,2015-07-01,2015-07-02,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,3,Resort Hotel,0,13,2015-07-01,2015-07-02,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,4,Resort Hotel,0,14,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
5,5,Resort Hotel,0,14,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
6,6,Resort Hotel,0,0,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.00,0,0,Check-Out,2015-07-03
7,7,Resort Hotel,0,9,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,NaN,0,Transient,103.00,0,1,Check-Out,2015-07-03
8,8,Resort Hotel,1,85,2015-07-01,2015-07-04,2015,July,27,1,0,3,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.00,0,1,Canceled,2015-05-06
9,9,Resort Hotel,1,75,2015-07-01,2015-07-04,2015,July,27,1,0,3,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.50,0,0,Canceled,2015-04-22




---



# **Prüfung der Datenqualität**



> Im nächsten Schritt kann nun der vorliegende Satz auf fehlende Werte überprüft werden. Hierzu eignet sich die Funktion `isna()`. `isna()` ermittelt alle fehlende Werte pro Spalte und Zeile. Wird `isna()` mit `sum()` verkettet, so wird die Gesamtanzahl fehlender Werte pro Spalte ausgegeben.



In [ ]:
# print sum of missing values per row
hotel_orig.isna().sum()

night_index                            0
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date                           0
departure_date                         0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
stay_nights_sum                        0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_ty



> Im Hotel-Datensatz sind die meisten Zellen mit Werten gefüllt, lediglich vier Spalten enthalten 'Missing Values' (children, country, agent und company). In der 'children'-Spalte fehlen 4 Angaben, unter 'country' fehlen bei 488 Datenzeilen die jeweiligen Werte, bei 'agent' sind es sogar 16340 und unter 'company' wurden in 112593 Zeilen keine Angaben vorgenommen.

> Durch Teilen der Summe aller fehlender Werte pro Spalte durch die Gesamtlände des Dataframes kann das Verhältnis fehlender Werte zu Gesamtwerten ermittelt werden. Wird dieses Ergebnis dann mit `apply('{0:.4%}'.format)` bearbeitet, wird es als Prozentwert ausgegeben.





In [ ]:
# print percentage of missing values per column
(hotel_orig.isna().sum()/len(hotel_orig)).apply('{0:.4%}'.format)

night_index                        0.0000%
hotel                              0.0000%
is_canceled                        0.0000%
lead_time                          0.0000%
arrival_date                       0.0000%
departure_date                     0.0000%
arrival_date_year                  0.0000%
arrival_date_month                 0.0000%
arrival_date_week_number           0.0000%
arrival_date_day_of_month          0.0000%
stays_in_weekend_nights            0.0000%
stays_in_week_nights               0.0000%
stay_nights_sum                    0.0000%
adults                             0.0000%
children                           0.0034%
babies                             0.0000%
meal                               0.0000%
country                            0.4087%
market_segment                     0.0000%
distribution_channel               0.0000%
is_repeated_guest                  0.0000%
previous_cancellations             0.0000%
previous_bookings_not_canceled     0.0000%
reserved_ro

> Die heraus stechenden Spalten sind, wie schon beschrieben, 'company' mit ~94.3% und 'agent' mit ~13.7%. Für die Spalten 'country' und 'children' wurde ein Prozentsatz an fehlenden Werten von ~0.4% bzw. 0.0034% errechnet. Da bei diesen Spalten die Anzahl fehlender Werte so gering sind, aber dennoch in die weiteren Analysen einfließen könnten, entfernen wir hier die Zeilen mit Missing Values.

In [ ]:
hotel_orig = hotel_orig.dropna(subset=['children', 'country'])
(hotel_orig.isna().sum()/len(hotel_orig)).apply('{0:.4%}'.format)

night_index                        0.0000%
hotel                              0.0000%
is_canceled                        0.0000%
lead_time                          0.0000%
arrival_date                       0.0000%
departure_date                     0.0000%
arrival_date_year                  0.0000%
arrival_date_month                 0.0000%
arrival_date_week_number           0.0000%
arrival_date_day_of_month          0.0000%
stays_in_weekend_nights            0.0000%
stays_in_week_nights               0.0000%
stay_nights_sum                    0.0000%
adults                             0.0000%
children                           0.0000%
babies                             0.0000%
meal                               0.0000%
country                            0.0000%
market_segment                     0.0000%
distribution_channel               0.0000%
is_repeated_guest                  0.0000%
previous_cancellations             0.0000%
previous_bookings_not_canceled     0.0000%
reserved_ro



> Die Spalte 'company' beschreibt dabei Geschäftskunden bzw. Kunden, die über ihr Unternehmen gebucht haben. Trotz der vielen Missing Values, ist diese Spalte für das Projekt allerdings zweitrangig, weshalb die Werte zu vernachlässigen sind.

> In der Spalte 'agent' konnten ebenfalls, im Vergleich zu anderen Spalten, viele Missing Values verzeichnet werden. In dieser Spalte wurden Buchungen festgehalten, die über ein Reisebüro getätigt wurden. Da auch diese Information für das Projekt zweitrangig ist, sind die fehlenden Werte zu vernachlässigen.

> Dadurch, dass lediglich vier Spalten Missing Values enthalten und diese Spalten für die geplanten Anaylsen eher zweitrangig sind, lässt sich darauf schließen, dass der Datensatz für das Projekt gut geeignet ist.

- Agent sind Personen, die über ein Reisebüro gebucht haben 

- company und Agent für unsere Analysen eher zweitrangig, deswegen egal dass so viele Missing Values



## **Überprüfen auf Logikfehler**

> Bei der Verzeichnung der Buchungen kann es möglicherweise zu Logikfehlern gekommen sein. Daher überlegen wir uns, welche Situationen unrealistisch und damit ein Datenfehler sein müssen. 

> Bei den Buchungen sind folgende Fälle ein Indiz für fehlerafte Daten: 

> (1) In einer Buchung sind weder Übernachtungen unter der Woche noch am Wochenende verzeichnet.

> (2) Eine Buchung zeigt eine negative Lead Time vor, das Buchungsdatum befindet sich demnach nach dem Ankunftsdatum.

> (3) Als Gäste sind Kinder, aber keine Erwachsenen, verzeichnet.

> (4) Als Gäste sind Babies, aber keine Erwachsenen, verzeichnet.

> (5) Weder Erwachsene noch Kinder noch Babies sind als Gäste verzeichnet.

> (6) Jemand ist zum wiederholten Mal Gast, hat aber bisher weder eine Buchung storniert noch wahrgenommen. 

> Im Folgenden prüfen wir, ob die einzelnen Fälle in unserem Datensatz vorliegen und ziehen ggf. die notwendigen Konsequenzen. 

> **(1) In einer Buchung sind weder Übernachtungen unter der Woche noch am Wochenende verzeichnet.**

> Auffällig ist, dass schon in den ersten Zeilen Am selben Tag gebucht wurde, an dem die Person(en) auch eingecheckt, aber keine Nacht im Hotel verbracht haben. 

> Als erster Schritt der Prüfung auf einen Fehler bei der Eingabe wird ausgegeben, wie viele Buchungen vorliegen, bei denen Personen keine Nacht (weder am Wochenende noch unter der Woche) im Hotel verbracht haben. Dafür wird die `len()`-Funktion verwendet, die die Länge des Datensatzes, bzw. Anzahl an Zeilen, angeben soll, die nur Buchungen listet, bei denen unter 'stays_in_week_nights' und gleichzeitig unter 'stays_in_weekend_night' 0 angegeben ist. Durch Anwendung dieser Condition und Ausgabe des Ergebnisses wird erkenntlich, dass dies bei 715 Personen der Fall ist.

> Es scheint nicht sehr wahrscheinlich, dass bei 715 Buchungen ein zufälliger Fehler unterlaufen ist.





In [ ]:
# print sum of zero-nights spent at a hotel but booked anyway
num_zero_stays = len(hotel_orig[(hotel_orig['stays_in_week_nights'] == 0) & (hotel_orig['stays_in_weekend_nights'] == 0)])
num_zero_stays



> Möglich ist auch, dass der Besuch des Hotels aufgrund der Nutzung bestimmter Annehmlichkeiten als Buchung aufgeführt wird, ohne dass geplant ist, über Nacht zu bleiben. So könnte z.B. ein Spa- oder Restaurant-Besuch im Hotel ebenso in den Buchung aufgeführt sein, wie eine gebuchte Übernachtung. 

> Es wird nun die Annahme geprüft, dass besonders im Resort Hotel solche Buchungen vorliegen. Dafür wird die Condition von oben nochmals auf den Datensatz angewandt und mit dem Zusatz `'.hotel'` nur die Hotelnamen solcher Buchungen ausgegeben. Durch das voranstellen der `len()`-Funktion kann dann die genaue Zahl ermittelt werden, in der 0-Zero-Night-Stays in den beiden Hotels vorliegen.

> Es scheinen diese Art von Buchungen in beiden Hotels ähnlich häufig vorzuliegen. Entweder bietet also auch das City Hotel Annehmlichkeiten, die man auch als 'Nicht-Hotelgast' buchen kann, oder es liegt ein Logikfehler vor.







In [ ]:
# print hotel in which bookings with zero nights were made
hotel_zero_stays = hotel_orig[(hotel_orig['stays_in_week_nights'] == 0) & (hotel_orig['stays_in_weekend_nights'] == 0)].hotel
hotel_zero_stays

print(len(hotel_zero_stays[hotel_zero_stays == 'Resort Hotel']))
print(len(hotel_zero_stays[hotel_zero_stays == 'City Hotel']))

> **(2) Eine Buchung zeigt eine negative Lead Time vor.**

> Im Fall einer negativen Lead Time müsste das Buchungsdatum chronologisch nach dem Ankunftsdatum liegen. 

> Wir prüfen, bei wie vielen Zeilen diese Problematik auftritt und stellen fest, dass wir in unserem Datensatz keine negativen Lead Times vorfinden.

In [ ]:
# number of rows with a lead time below 0
len(hotel_orig[(hotel_orig['lead_time']<0)])

0

> **(3) Als Gäste sind Kinder, aber keine Erwachsenen, verzeichnet.**

> Ein Hotelaufenthalt ist in der Regel nicht für Kinder ohne Begleitung einer volljährigen Person gestattet. Falls sich in unserem Datensatz also Zeilen befinden, bei denen Kinder, aber keine Erwachsenen verzeichnet sind, handelt es sich möglicherweise um einen Datenfehler. 

> Wir stellen durch das Anwenden der `len()`-Funktion fest, dass es insgesamt 223 Zeilen in unserem Datensatz gibt, bei denen dieser Fall zutrifft. 

In [ ]:
# number of rows with children above 0 and 0 adults
len(hotel_orig[(hotel_orig['adults'] == 0) & (hotel_orig['children'] != 0)])

223

> Allerdings können wir nicht mit Sicherheit sagen, dass Eltern nicht die Buchung anstelle ihrer Kinder vornehmen, diese aber alleine im Hotel übernachten. Eine Buchung könnte also ausschließlich über eine erwachsene Person möglich sein, der Hotelaufenthalt aber auch für unbegleitete Minderjährige.

> Wir entscheiden uns, die Zeilen nicht rauszunehmen, sondern im weiteren Verlauf in unsere Analysen einzubeziehen. 

> **(4) Als Gäste sind Babies, aber keine Erwachsenen, verzeichnet.**

> Eine Buchung, bei der zwar ein oder mehrere Babies, aber keine Erwachsener verzeichnet, ist nicht realistisch. 

> Mithilfe der `len()`-Funktion stellen wir fest, dass dieser Fall auf 3 Zeilen unseres Datensatzes zutrifft. 

In [ ]:
# number of rows with 0 adults but babies above 0
len(hotel_orig[(hotel_orig['adults'] == 0) & (hotel_orig['babies'] > 0)])

3

> Wir gehen in diesem Fall von fehlerhaften Daten aus und entfernen die entsprechenden Zeilen aus dem Datensatz.

In [ ]:
# removing the rows with 0 adults but babies above 0
hotel_orig = hotel_orig.loc[~((hotel_orig['adults'] == 0) & (hotel_orig['babies'] > 0))]

> **(5) Weder Erwachsene noch Kinder noch Babies sind als Gäste verzeichnet.**

> In diesem Fall besteht eine Buchung, allerdings sind keine Gäste verzeichnet.

> Verwenden wir wieder die `len()`-Funktion sehen wir, dass genau 180 Zeilen diese Eigenschaft vorweisen.

In [ ]:
# number of rows with 0 adults, 0 children and 0 babies
len(hotel_orig[(hotel_orig['adults'] == 0) & (hotel_orig['children'] == 0) & (hotel_orig['babies'] == 0)])

180

> Da es sich hier um fehlerhafte Daten handeln muss, entfernen wir die betroffenen Zeilen aus unserem Datensatz.

In [ ]:
# removing the rows with 0 adults, 0 children and 0 babies
hotel_orig = hotel_orig.loc[~((hotel_orig['adults'] == 0) & (hotel_orig['children'] == 0) & (hotel_orig['babies'] == 0))]

> **(6) Jemand ist zum wiederholten Mal Gast, hat aber bisher weder eine Buchung storniert noch wahrgenommen.**

> In diesem letzten Fall hat ein Gast zum wiederholten Mal eine Buchung vorgenommen. Es wird zusätzlich angegeben, wie viele der vorherigen Buchungen storniert wurden und wie viele nicht storniert wurden. Wenn ein Gast also bereits in der Vergangenheit eine Buchung vorgenommen hat, muss er diese entweder wahrgenommen oder storniert haben.

> Wir wollen jetzt herausfinden, wie viele Zeilen esgibt, in denen ein wiederholter Gast bisher weder eine Buchung wahrgenommen noch storniert hat. Wir verwenden hierfür die `len()`-Funktion und sehen, dass insgesamt 590 Zeilen betroffen sind.

In [ ]:
# number of rows with repeated guest equal to 1, but both previous cancellations and not-cancelled bookings equal to 0
len(hotel_orig[(hotel_orig['is_repeated_guest'] == 1) & (hotel_orig['previous_cancellations'] == 0) & (hotel_orig['previous_bookings_not_canceled'] == 0)])

0

> Wir entscheiden uns, auf diesen Logikfehler mit dem Entfernen der betroffenen Zeilen zu reagieren.

In [ ]:
# removing the rows with repeated guest equal to 1, but both previous cancellations and not-cancelled bookings equal to 0
hotel_orig = hotel_orig.loc[~((hotel_orig['is_repeated_guest'] == 1) & (hotel_orig['previous_cancellations'] == 0) & (hotel_orig['previous_bookings_not_canceled'] == 0))]

# **Joinen mit Wetterdaten**

## **Vorbereiten des Wetter-Datensatzes**



> Nun soll der Hotel-Datensatz mit Wetterdaten verknüpft werden, um den Einfluss des Wetters auf die Hotelauslastung bzw. Cancellation von Buchungen ermitteln zu können. Dafür wird Meteostat installiert und die Funktionen `Stations()` und `Daily()` importiert.



In [ ]:
pip install meteostat

In [ ]:
from meteostat import Stations, Daily

### **Wetterdaten Lissabon**



> Mit der in `meteostat` hinterlegten `Stations()`-Funktion können wir mit `Stations().nearby(x,y)` und den Koordinaten von Lissabon die nächststehende Wetterstation von Lissabon ermitteln. Diese liegt direkt in Lissabon selbst, da die Koordinaten (im Output) sich nur minimal von der Eingabe unterscheiden. Die Station wird unter 'station_lisbon' gespeichert. Der wichtige Output dieses Codeabschnitts ist die ID der relevanten Wetterstation. Diese wird in den folgenden Schritten angegeben um sich auf genau diese Wetterstation zu beziehen.



In [ ]:
# get weather station near lisbon
stations = Stations()
station_lisbon = stations.nearby(38.7167, -9.1333) 
station_lisbon = station_lisbon.fetch(1)

print(station_lisbon)

Nun werden den Variablen 'start' und 'end' der Zeitraum für die Beobachtungen im Hotel Datensatz übergeben. Anschließend wird mit der Funktion `Daily()` und den Parametern Wetterstation (die ID, die oben gerade ermittelt wurde) und Zeitraum, die täglichen Wetterdaten für Lissabon unter der Variable 'weather_lisbon gespeichert. Dann können über `df.fetch()` die Wetterdaten von meteostat eingeholt werden.[Linktext](https://)

In [ ]:
# Import dependencies
from datetime import datetime

# Set time period
start = datetime(2015, 7, 1)
end = datetime(2017, 8, 31)

# Get daily data
weather_lisbon = Daily('08535', start, end)
weather_lisbon = weather_lisbon.fetch()



> Mit der `mathplotliberary` können nun die Wetterdaten für einen bestimmten Zeitraum (hier: 01.07.2015-31.08.2017) als Liniendiagramm dargestellt werden. Anhand dieser Daten können die Temperaturentwicklungen über den gewählten Zeitraum eingesehen werden.

In [ ]:
# Import Meteostat library
import matplotlib.pyplot as plt

# Plot line chart including average, minimum and maximum temperature
weather_lisbon.plot(y=['tavg']) #, 'tmin', 'tmax'
plt.show()

In [ ]:
weather_lisbon



> Um einen Überblick über die Wetterdaten zu erhalten, können diese außerdem in einem Dataframe und mit `describe()` die wichtigsten statistischen Daten ausgegeben werden.



In [ ]:
weather_lisbon.describe()



> Um später Bereinigungen und Analysen zu vereinfachen, muss an dieser Stelle der Index, der momentan als Datum definiert ist, zurückgesetzt werden. So kann später einfach über die Standard-Indexierung [0:...] auf einzelne Zeilen zugergiffen werden.

In [ ]:
weather_lisbon = weather_lisbon.reset_index()



> Außerdem muss innerhalb der Daten kenntlich gemacht werden, dass dies die Wetterdaten von Lissabon sind. Hierfür fügen wir eine neue Spalte 'city' ein, die wir für den kompletten Datensatz mit 'Lisbon' füllen. Wird dieser Datensatz dann mit einem anderen Wetterdatensatz gejoint bleibt kenntlich, welche Daten sich auf welchen Ort beziehen.



In [ ]:
weather_lisbon.insert(loc=0, column='city', value ='Lisbon')
weather_lisbon

### **Wetterdaten Faro**



> Wie bei den Wetterdaten für Lissabon, kann mit `Stations()`-Funktion eine Wetterstation in der Algarve ermitteln. Als Koordinaten, in deren Nähe die Station liegen soll, werden die von Faro, der Hauptstadt der Algarve angegeben. Die ermittelte Station liegt somit in der Nähe von Faro und wird unter 'station_faro' gespeichert.



In [ ]:
# get data of a weather station located in the algarve
station_faro = stations.nearby(37.0187, -7.9272) 
station_faro = station_faro.fetch(1)

print(station_faro)



> Es kann nun wieder die gewünschte Station und der gewünschte Zeitraum in die `Daily()`-Funktion gegeben und die entsprechenden Wetterdaten eingeholt und geplottet werden.



In [ ]:
# Get daily data
weather_faro = Daily('08554', start, end)
weather_faro = weather_faro.fetch()

# Plot line chart including average, minimum and maximum temperature
weather_faro.plot(y=['tavg']) #, 'tmin', 'tmax'
plt.show()



> Auch ein Überblick über die grundlegenden statischtischen Kennzahlen der Wetterdaten in Faro kann mit `describe()` ausgegeben werden.



In [ ]:
weather_faro.describe()



> Um zu prüfen, wie der Wetterdatensatz für Faro bereinigt werden muss, wird dieser durch Eingabe und Ausführen des Dataframe-Namens 'weather_faro' als Dataframe ausgegeben.



In [ ]:
weather_faro



> Da der Datensatz momentan (wie bei dem für Lissabon auch schon) über das Datum indexiert ist, was später problematisch werden könnte, wird der Index über `reset_index()` zurückgesetzt. Die Änderung wird durch Ausgabe des Dataframes erneut überprüft.



In [ ]:
weather_faro = weather_faro.reset_index()
weather_faro



> Ebeneso muss der Stadtname 'Faro' wieder in einer neuen Spalte angefügt werden, um die Daten später noch zuordnen zu können.



In [ ]:
weather_faro.insert(loc=0, column='city', value ='Faro')
weather_faro

### **Zusammenführen der Wetterdatensätze**



> Es können nun die beiden vorbereiteten Datensätze mithilfe der pandas-Funktion `concat()` untereinander zusammengeführt werden. Die Daten werden untereinander vereint, da in beiden Datensätzen die Spaltennamen identisch sind.



In [ ]:
weather = pd.concat([weather_lisbon, weather_faro])
weather



> Da für die Vorhersage von Buchundsstornierungen und der Auslastung der Hotels Angaben wie Schnee, Wind und Luftdruck eine vernachlässigbare Rolle spielen und der Wetter-Datensatz für Lissabon in diesen Spalten viele NaN-Values hat (diese Daten also nicht vorhanden sind), können die entsprechenden Spalten über die '`drop()`'-Funktion gelöscht werden.



In [ ]:
weather = weather.drop(columns=['wdir', 'wspd', 'pres', 'snow', 'wpgt', 'tsun'])

## **Zusammenführen der konkatenierten Wetterdatensätze und des Hoteldatensatzes**



> Sobald die Wetterdatensätze zusammengeführt und irrelevante Spalten entfernt wurden, können diese datensätze mit dem Hoteldatensatz zusammengeführt werden. Dabei gilt beim Hoteldatensatz (left) die Spalte 'arrival_date', beim kombinierten Wetterdatensatz die Spalte 'time' als Indikator für das Verbinden der richtigen Wetterdaten mit der passenden Hotelbuchungen. Fehlen Wetterdaten zu den Hotelbuchungen, so werden die Hotelbuchungen trotzdem aufgeführt, die fehlenden Daten allerdings durch 'NaN' ersetzt (argument: how='left').



In [ ]:
hotel_and_weather = pd.merge(hotel_orig, weather, left_on='arrival_date', right_on='time', how='left')
hotel_and_weather



> Durch Ausgeben des zusammengeführten Datensatzes wird jedoch deutlich, dass sich die Anzahl der Einträge verdoppelt hat. Dies liegt daran, dass Wetterdaten für jeden Tag sowohl für Lissabon, als auch für Faro vorliegen. Durch das Anwenden der `merge()`-Funktion wurden nun zu jedem 'Arrival-Date' die Wetterdaten beider Städte hinzugefügt. Um dies zu beheben, werden nun die Zeilen gesucht, in denen sowohl 'Resort Hotel' als auch 'Faro' vorhanden sind (da laut der Dokumentation des Datensatzes das Resort Hotel in der Algarve liegt) ODER das Hotel ein 'City Hotel' ist und mit den Wetterdaten von 'Lissabon' verbunden wurden. Durch das Angeben der unter 'condition' gespeicherten Bedingungen in eckigen Klammern [] nach dem Dataframe-Namen, werden ebendiese Reihe gefiltert und ausgegeben.



In [ ]:
condition = ((hotel_and_weather['hotel'] == 'Resort Hotel') & (hotel_and_weather['city'] == 'Faro') | 
               (hotel_and_weather['hotel'] == 'City Hotel') & (hotel_and_weather['city'] == 'Lisbon'))

hotel_and_weather_filtered = hotel_and_weather[condition]
hotel_and_weather_filtered



> Nun müssem erneut die Indices zurückgesetzt werden, da diese nicht in aufsteigender Reihenfolge vergeben sind (da Zeilen gestrichen wurden). Die Spalte, die unter 'index' aufgeführt ist doppelt sich und auch die Spalte 'time', über die die Wetterdaten mit den Hoteldaten verbundden wurde, liegt doppelt vor. Diese beiden Spalten können nun entfernt werden. 



In [ ]:
hotel_and_weather_filtered = hotel_and_weather_filtered.reset_index()
hotel_and_weather_filtered = hotel_and_weather_filtered.drop(columns = ['index', 'time'])
hotel_and_weather_filtered

> Da für weitere Analysen jeweils ausreicht zu wissen, wie die durchschnittliche Mindest- und Höchsttemperatur, die Temperatur im aritmetischen Mittel und der durchschnittliche Niederschlag im Buchungszeitraum aussieht, sollen diese Durchschnitsswerte nun in einem Dataframe zusammengefasst werden.

> Dafür wird zuerst eine leere Liste (= holidaaverage_weathery_flag) erstellt. Für jeden Index bzw. jede Zeile der hotel_and_weather_filtered (also jede Beobachtung) kann nun iteriert und geprüft werden, ob die Condition (=filtered_df), dass die Wetterdaten sich auf das Datum nach der Ankunft und vor der Abfahrt sowie der entsprechenden Stadt bezieht. 

> Nach Anwenden der 'Filterung' auf die einzelnen Reihen, können nun die einzelnen arithmetischen Mittelwerte für die tagesdurchschnittliche Temperatur, die Mindest- und Höchstetemperaturen und den Niederschlag über den Buchungszeitraum hinweg berechnet werden.
Diese Daten werden einzeln als Variablen gespeichert und jeweils inklusive der Städtenamen in der average_weather-Liste angehängt. 

>Zum Schluss kann die average_weather-Liste in eine Dataframe transformiert werden.

In [ ]:
average_weather = []

# Durchschnittliche Wetterdaten pro Stadt im betrachteten Zeitraum berechnen
for index, row in hotel_and_weather_filtered.iterrows():
    city = row['city']
    arrival = row['arrival_date']
    departure = row['departure_date']
    
    filtered_df = weather[(weather['city'] == city) & (weather['time'] >= arrival) & (weather['time'] <= departure)]
    average_temp = filtered_df['tavg'].mean()
    average_tmin = filtered_df['tmin'].mean()
    average_tmax = filtered_df['tmax'].mean()
    average_prcp = filtered_df['prcp'].mean()

    
    average_weather.append({
        'city': city,
        'average_temperature': average_temp,
        'average_temp_min': average_tmin, 
        'average_temp_max': average_tmax,
        'average_prcp': average_prcp,

    })

average_weather_df = pd.DataFrame(average_weather)



> Um den average_weather-Datensatz mit dem hotel_and_weather-Datensatz zu joinen benötigen wir nun noch die night_index-Spalte, um den Join der beiden Datensätze über diese identische Spalte zu ermöglichen.

> Dazu wird in den Dataframe 'average_wather' die neue Spalte 'night_index' eingefügtund durch die `range()`-Funktion, die eine aufsteigende Zahlenfolge von 0 bis zur Länge von stay_dates, über `len() `ermittelt, befüllt. Somit wird den Reihen von average_weather in der 'night_index'-Spalte die Werte 0-(len(stay_dates)-1) zugewiesen.





In [ ]:
average_weather_df['night_index'] = range(len(average_weather_df))
average_weather_df.loc[:, 'night_index'] = range(len(average_weather_df))

> Nun können die beiden Datensätze über `pd.merge()` miteinander gejoint werden.

In [ ]:
hotel_and_weather_filtered = pd.merge(hotel_and_weather_filtered, average_weather_df, on='night_index')

> Zuletzt können die Spalten 'city_x', 'tavg', 'tmin', 'tmax' und 'recp' durch die Funktion `drop()` entfernt werden, da diese nur zur Herleitung der relevanten Werte benötigt wurden.

In [ ]:
hotel_and_weather_filtered = hotel_and_weather_filtered.rename(columns={'city_y': 'city'})

> Zur Überprüfung, ob die Conditions bis zu diesem Punkt korrekt umgesetzt wurden und die gewünschten Ergebnisse liefern bzw. die Wetterdaten richtig zugeordnet sind, wird nun mit `nunique()` überprüft, wie viele einzigartige Werte pro Spalte vorliegen.

In [ ]:
hotel_and_weather_filtered.nunique()

# **Joinen mit Feiertagen**

## **Vorbereitung des Holiday-Datensatzes**



> Da die Vermutung naheliegt, dass es einen Einfluss auf Buchungen und Cancellations haben kann, ob der Buchungszeitraum an einem Feiertag liegt, wollen wir den kombinierten hotel_and_weather_filtered-Datensatz mit Daten zu nationalen Feiertagen zusammenführen.
Um zu ermitteln, welche Form die Werte der Spalte 'country' annehmen, um einen Join zu ermöglichen, schauen wir, in welcher Form die countries im hotel-and-weather-Datensatz vorliegen. Dies erreichen wir über die `unique()`-Funktion, die auf die country-Spalte angewendet wird. Diese einzeln vorliegenden Werte können wir dann mit `print()` ausgeben. Wir sehen, dass die Länder in Alpha 3-Codes vorliegen.



In [ ]:
unique_countries = hotel_and_weather_filtered['country'].unique()
print(unique_countries)



> Aus dem package 'datetime' muss 'date' importiert werden, um später das Datum im Holiday-Datensatz im gewünschten Format ausgeben zu können. Zudem wird das Paket 'holidays' importiert, welche die Feiertage aller relevanten Länder beinhaltet.



In [ ]:
from datetime import date
import holidays 



> Um jeweils das Datum der Feiertage in Deutschland für die einzelnen Jahre zu erhalten, wird eine leere Liste ('holiday_list') erstellt. Über eine for-Loop werden dafür für die einzelnen Feiertage in Deutschland* im Datensatz holidays aus den Jahren 2015, 2016 und 2017 gezogen. Dies geschieht über die condition 'years= 2015,2016, 2017'. Mit der Funktion `items()`, die Tuples aus den entsprechenden Feiertagen mit dem jeweiligen Datum erstellt.


> Diese Liste mit Tuples wird dann in ein Dataframe umgewandelt, mit den Spalten 'date' und 'holiday' versehen und das Dataframe unter 'holidays_df' gespeichert.

> Nach Erstellen dieses Dataframes wird es ausgegeben um einen Qualitätscheck zu machen. Deswegen wurden hier auch die deutschen Feiertage verwendet um die Überprüfung einfach zu gestatlten. 

'*  Hier wurden nur die Feiertage verwendet, die in ganz Deutschland einheitlich sind. Dies bedeutet, dass bundeslandspezifische Feiertage nicht berücksichtigt werden.


> Nun brauchen wir noch ein Dataframe aller Feiertage mit den entsprechenden Länderkürzeln, um die beiden Datensätze joinen zu können. Dennoch dient der Code als Ansatz, für alle Länder inklusive der Länderkürzel die entsprechenden Feiertage zu bekommen.



In [ ]:
holiday_list = []
for holiday in holidays.Germany(years=[2015,2016,2017]).items():
  holiday_list.append(holiday)

holidays_df = pd.DataFrame(holiday_list, columns=['date', 'holiday'])
holidays_df



> Dafür werden über die Funktion `list_supported_countries()`, die auf dem holiday-Datensatz angewendet wird, alle Ländercodes des holidays-packages, unter 'all_country_codes' gespeichert.

> Nun wird eine leere Liste 'holiday_list' erstellt, in der die Informationen, die durch zwei verschachtelte for-Loops ermittelt werden, abgelegt werden können.

> In einer ersten for-Loop wird angestoßen, dass für jedes einzelne Länderkürzel der ermittelten 'all_country_codes' ein bestimmter Befehl durchgeführt wird. Dieser Befehl ist, für den Countrycode des entsprechenden Landes die Feiertag-Daten aus den Jahren 2015-2017 zu ziehen*. Dafür wird im holidays-Datensatz in der Spalte 'CountryHoliday' unter der Bedingung, dass der Countrycode übereinstimmt und der Tag zwischen 2015 und 2017 liegt, das entsprechende Datum ermittelt und unter holiday_dates gespeichert.

> Innerhalb dieser holiday_dates-Tuples wird dann pro Datum und Feiertagname das Länderkürzel und entsprechende Datum an die leere holiday_list gehängt.



> Sobald all diese Daten und Codes in der Liste gesammelt sind, werden diese unter den Spalten 'country_code' und 'date' als Dataframe mit dem Namen 'holidays_df' gespeichert. Dieses kann dann zur Überprüfung ausgegeben werden.

'*  Für Deutschland wurden auch hier nur die Feiertage verwendet, die in ganz Deutschland einheitlich sind. Bundeslandspezifische Feiertage werden deshalb nicht berücksichtigt.










In [ ]:
# Get a list of all available country codes
all_country_codes = holidays.list_supported_countries()

# Create an empty list to store holiday information
holiday_list = []

# Loop through each country code and append its holiday information to the list
for code in all_country_codes:
    # Retrieve the holiday dates for the country and year range
    holiday_dates = holidays.CountryHoliday(code, years=[2015, 2016, 2017])
    
    # Loop through each holiday date and append the country name and date to the list
    for date, name in holiday_dates.items():
        holiday_list.append((code, date))


holidays_df = pd.DataFrame(holiday_list, columns=['country_code', 'date'])
holidays_df



> Mit der Funktion `.dtype` kann dann der Datentyp der Spalte 'date' im 'holiday_df' ermittelt werden.



In [ ]:
print(holidays_df['date'].dtype)



> Nun kann durch `pd.to_datetime` die date-Spalte in das gewünschte Format (yyyy-mm-dd) gebracht werden. Durch speichern unter dem gleichen Namen, wird das Dataframe aktualisiert abgespeichert.



In [ ]:
holidays_df['date'] = pd.to_datetime(holidays_df['date']) # date Spalte zu datetime format


> Durch erstellen einer Maske, die aus zwei mit einem 'and' verbundenen Conditions besteht, kann sichergestellt werden, dass nur Feiertage im Dataframe enthalten bleiben, die innerhalb der gewünschten Zeitperiode liegen. Diese Zeitperiode wurde bereits bei der Vorbereitung der Wetterdatensätze erstellt (und unter 'start' und 'end' gespeichert) und kann hier wiederverwendet werden.



In [ ]:
mask = (holidays_df['date'] >= start) & (holidays_df['date'] <= end)
holidays_df = holidays_df.loc[mask]
holidays_df

In [ ]:
pip install pycountry



> Die Datensätze können allerdings immer noch nicht gejoint werden, da im holiday_and_weather-Datensatz die Daten in einem Alpha 3-Code vorliegen, im holidays-Datensatz ist es ein Alpha 2-Code. Dies muss also angeglichen werden.
Dafür benötigen wir den 'country_converter', der installiert und als 'coco' importiert wird.



In [ ]:
pip install country_converter

In [ ]:
import country_converter as coco



> Für eine Umwandlung von Alpha 2- in Alpha 3-Codes, wird eine Liste der Länderkürzel benötigt. Dafür erstellen wir über die Funktion `tolist()` eine Liste aller im holidays-Datensatz in der Spalte 'country_code' enthaltenen Werte eine Liste.



In [ ]:
code_list = holidays_df['country_code'].tolist()
print(code_list)


> Als 'alpha3' kann mit dem country_converter und der Funktion `convert()` die in der code_list enthaltenen Länderkürzel in das ISO3-Format gebracht werden, und haben somit 3 Buchstaben im Kürzel. Über den Parameter 'not_found=None' wird definiert, dass für nicht gefundene Werte das Originalkürzel beibehalten wird.



In [ ]:
alpha3 = coco.convert(names=code_list, to='ISO3', not_found=None)
print(alpha3)



> Die als 'alpha3' gespeicherten Länderkürzel werden nun als neue Spalte 'alpha3_code' dem holidays-Dataframe angehängt. Das holidays-Dataframe hat nun die drei Spalte 'date', 'country_code' und 'alpha3_code' und kann auch so ausgegeben werden.



In [ ]:
holidays_df['alpha3_code'] = alpha3
holidays_df = holidays_df[['date', 'country_code', 'alpha3_code']]
holidays_df

> Durch die Funktion '`nunique()`' kann nun ermittelt werden, wie viele unterschiedlichen Antworten gegeben wurden. Da die Daten sich auf zwei verschiedene Hotels beziehen, gibt es in der Spalte 'hotel' nur zwei unterschiedliche Werte unter allen Beobachtungen.

In [ ]:
hotel_orig.nunique()



> Da das Joinen des holidays-Datensatzes zum holiday_and_weather-Datensatz dazu führen soll, dass ein Abgleich möglich ist, ob Buchungen an/um Feiertage(n) liegen, müssen die Ankunfts- und Abreisdaten aus dem hotel-Datensatz extrahiert werden. Dies geschieht durch das Speichern der einzelnen Spalten 'arrival_date', 'depature_date' und 'country' unter dem Variablennamem 'stay_dates'


In [ ]:
stay_dates = hotel_orig[['arrival_date', 'departure_date', 'country']]
stay_dates



> Da für weitere Analysen ausreicht zu wissen, ob sich der Buchungszeitraum mit einem oder mehreren Feiertagen überschneidet, genüg eine Angabe mit 0- und 1-Werten (=Holiday-Flag). 

> Für die Holiday-Flags wird zuerst eine leere Liste (= holiday_flag) erstellt. Für jeden Index bzw. jede Zeile der stay_dates kann nun iteriert und geprüft werden, ob die Condition (=mask), dass ein Feiertag des holidays-Dataframe nach dem Datum der Ankunft und vor der Abfahrt aus stay_dates liegt. Die Condition beinhaltet außerdem, dass das Länderkürzel in stay_dates mit dem in holidays_df übereinstimmt, die Feiertage also wirklich die des jeweiligen Landes sind.

> Nach Anwenden der Maske auf die einzelnen Reihen in stay_dates, kann durch die `len()`-Funktion die Anzahl der Feiertage bestimmt werden, die sich mit dem Buchungszeitraum überschneiden. Diese Anzahl wird unter 'nmbr_holidays' gespeichert.

> Nun kann geprüft werden, ob die Anzahl der Feiertage, die sich mit dem Buchungszeitraum überschneiden, größer als null ist. Ist dies der Fall (überschneidet sich der Buchungszeitraum also mit einem Feiertag), wird in einer neuangefügten Spalte 'holiday_flag' im stay_dates-Dataframe eine Flag (=1) gesetzt. Alle anderen Buchungszeiträume erhalten keine Holiday-Flag.









In [ ]:
holiday_flag = [] 

for index, row in stay_dates.iterrows():
  arrival_date = row['arrival_date']
  departure_date = row['departure_date']
  country = row['country']

  mask = (holidays_df['date'] >= arrival_date) & (holidays_df['date'] <= departure_date) & (holidays_df['alpha3_code'] == country)

  nmbr_holidays = len(holidays_df[mask])

  if nmbr_holidays > 0: 
    holiday_flag.append(1)
  else: 
    holiday_flag.append(0)

stay_dates['holiday_flag'] = holiday_flag
stay_dates



> Das aktuelle stay_dates-Dataframe wird nun auf Duplikate überprüft. Die print-Ausgabe zeigt, dass 81.066 Zeilen Duplikate sind.



In [ ]:
# Count the number of duplicate rows
duplicate_rows = stay_dates.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"Number of duplicate rows: {num_duplicates}")

> Mit der Funktion '`value_counts()`', die bestimmt, wie häufig ein bestimmter Wert pro Spalte vorkommt, kann nun auch ermittelt werden, wie oft sich ein Aufenthalt im Hotel mit Feiertagen überschnitten hat. Es fällt auf, dass im Verhältnis nur sehr wenige Hotelaufenthalte sich mit einem nationalen Feiertag überschnitten haben.

In [ ]:
stay_dates.holiday_flag.value_counts()

## **Zusammenführen des Holiday- und Main-Datensatzes**


> Das letzte zu lösende Problem ist nun, dass stay_dates die 'night_idex'-Spalte als eindeutige ID-Spalte für den Join mit dem hotel_and_weather-Datensatz benötigt. Diese Spalte muss nun generiert werden.

> Auch hier wird in den Dataframe 'stay_dates' eine neue Spalte 'night_index' eingefügt.und mit den Index-Werten des Dataframes 'stay_dates' befüllt. Dies wird durch die `range()`-Funktion, die eine aufsteigende Zahlenfolge von 0 bis zur Länge von stay_dates, über `len() `ermittelt. Somit wird den Reihen von stay_dates in der 'night_index'-Spalte die Werte 0-(len(stay_dates)-1) zugewiesen.



In [ ]:
stay_dates['night_index'] = range(len(stay_dates))
stay_dates.loc[:, 'night_index'] = range(len(stay_dates))
stay_dates



> Nun kann der stay_dates-Datensatz nach den Holiday-Flags und dem Night-Index gefiltert werden, den beiden für den Join bzw. die Analysen notwendigen Spalten.



In [ ]:
stay_dates_filtered = stay_dates[['holiday_flag', 'night_index']]
print(stay_dates_filtered)



> Zuletzt kann der zusammengeührte hotel_and_weather-Datensatz über `pd.merge()` mit dem stay_dates_filtered-Datensatz gejoint werden. Dies geschieht über die 'night_index'-Spalte. Der nun vollständig angereicherte Datensatz wird als Variable 'hotel_and_weather_and_holidays' gespeichert.



In [ ]:
hotel_and_weather_and_holidays = pd.merge(hotel_and_weather_filtered, stay_dates_filtered, on='night_index')
hotel_and_weather_and_holidays